<a href="https://colab.research.google.com/github/HireTheHero/Machine_Notebooks/blob/main/vilioexample_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# villo walkthrough
- Environment: colab pro
- Reproducing official procedure in [SCORE_REPRO.md](https://github.com/Muennighoff/vilio/blob/master/SCORE_REPRO.md)

## Prerequisite (Google Drive)
- Add kaggle.json to MyDrive/vilio
- Add hateful memes challenge data as MyDrive/vilio/hateful_memes.zip

## how to reproduce
### General
- Dependent on which model to run, comment in/out codes in `init / data`, `prep features`, `additional installation`, `hm_pipeline` (for including-training protocol) & `inference-only` (for inference-only protocol) subsections

### Including-training
- Click `Run All`
  - Allow access to MyDrive
  - Resume (from rest of init to hm_pipeline requirements installation)
  - At the end of `additional installation`, the notebook will stop for required restart
- Run again from `download model`
- Run the notebook for 5 times (O/U/V/ES/EL)
  - For runs among O/U/V or among ES/EL, resuming from `download model` should be fine
  - When you switch between O/U/V and ES/EL, running the whole thing will be needed
    - features will be copied to different location
- Comment in final scoring subsection and run again

### inference-only
- Coment out a cell with `comment out for inference-only` description
- Run till the end of `data`
- Run `additional installation`
- Run `inference-only` 



### TBD
- Add final scoring section
- Reproduce all of the procedure

## init

In [2]:
# mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### installation

In [3]:
# vilio basics
!cd ../; git clone -b master https://github.com/Muennighoff/vilio.git
!cd ../vilio/py-bottom-up-attention; pip install -r requirements.txt
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!cd ../vilio/py-bottom-up-attention; python setup.py build develop

Cloning into 'vilio'...
remote: Enumerating objects: 2834, done.
remote: Total 2834 (delta 0), reused 0 (delta 0), pack-reused 2834
Receiving objects: 100% (2834/2834), 10.61 MiB | 13.58 MiB/s, done.
Resolving deltas: 100% (1492/1492), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-ar9t052x
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-ar9t052x
     |████████████████████████████████| 753.4 MB 7.2 kB/s 
     |████████████████████████████████| 4.0 MB 32.3 MB/s 
     |████████████████████████████████| 42 kB 821 kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-py3-none-any.whl size=65332 sha256=599add1637843bca1141133c8e7fcf7f9371f71ebe37784b5ccd8dcc6bf6f98b
  Stored in directory: /tmp/pip-ephem-wheel-cache-m53ee21y/wheels/24/1d/09/8167de727fe5b74f832b6fcb5d9069d8f03ca29f337bfe484d
  Created wheel 

In [4]:
# for lmdb feats
!pip install -q kaggle

### kaggle setup
- [reference](https://www.kaggle.com/general/74235)

In [5]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/vilio/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
kaggleashwin/population-dataset                                World Population by Countries Dataset (1960-2021)    69KB  2022-08-31 05:30:26            785         35  1.0              
iamsouravbanerjee/house-rent-prediction-dataset                House Rent Prediction Dataset                        82KB  2022-08-20 13:49:03           7378        212  1.0              
iamsouravbanerjee/world-population-dataset                     World Population Dataset                             17KB  2022-08-31 11:20:04           1791         79  1.0              
sveta

### data
- [kaggle reference](https://qiita.com/k_ikasumipowder/items/1c20d8b68dbc94ab2633)

In [6]:
# hateful memes from MyDrive
!mkdir tmp_data
!unzip /content/drive/MyDrive/vilio/hateful_memes.zip -d tmp_data


ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: tmp_data/hateful_memes/img/58479.png  
  inflating: tmp_data/hateful_memes/img/58490.png  
  inflating: tmp_data/hateful_memes/img/58491.png  
  inflating: tmp_data/hateful_memes/img/58497.png  
  inflating: tmp_data/hateful_memes/img/58601.png  
  inflating: tmp_data/hateful_memes/img/58602.png  
  inflating: tmp_data/hateful_memes/img/58604.png  
  inflating: tmp_data/hateful_memes/img/58607.png  
  inflating: tmp_data/hateful_memes/img/58609.png  
  inflating: tmp_data/hateful_memes/img/58612.png  
  inflating: tmp_data/hateful_memes/img/58613.png  
  inflating: tmp_data/hateful_memes/img/58621.png  
  inflating: tmp_data/hateful_memes/img/58631.png  
  inflating: tmp_data/hateful_memes/img/58637.png  
  inflating: tmp_data/hateful_memes/img/58640.png  
  inflating: tmp_data/hateful_memes/img/58642.png  
  inflating: tmp_data/hateful_memes/img/58649.png  
  inflating: tmp_data/hateful_memes/img/58672.png  
  inflating: tmp_data/hateful_me

In [7]:
# hateful memes from MyDrive
## O/U/V
# !cp -r tmp_data/hateful_memes/* /vilio/data/
## E
!cp -r tmp_data/hateful_memes/* /vilio/ernie-vil/data/hm/

In [8]:
# kaggle lmdb
!kaggle datasets download muennighoff/hmfeatureszipfin

100% 9.58G/9.58G [02:16<00:00, 122MB/s]
100% 9.58G/9.58G [02:16<00:00, 75.2MB/s]


In [9]:
# kaggle lmdb
## O/U/V
# !unzip hmfeatureszipfin.zip -d /vilio/data/
## E
!unzip hmfeatureszipfin.zip -d /vilio/ernie-vil/data/hm/

Archive:  hmfeatureszipfin.zip
  inflating: /vilio/ernie-vil/data/hm/detectron.lmdb/data.mdb  
  inflating: /vilio/ernie-vil/data/hm/detectron.lmdb/lock.mdb  


In [ ]:
# hateful memes from MyDrive (for feature extraction)
## comment out for inference-only
# !cp -r tmp_data/hateful_memes/* /vilio/py-bottom-up-attention/data

In [10]:
## delete original for memory saving
!rm -r tmp_data

In [11]:
# kaggle lmdb
## delete original for memory saving
!rm hmfeatureszipfin.zip

## prep features

### extract image features
- [author's reference](https://www.kaggle.com/muennighoff/vilioexample-nb)

In [ ]:
# !cp -r /content/drive/MyDrive/vilio/sample/* ../vilio/py*/data/

In [ ]:
# # hm_feats.sh (O/U/V)
# ## Attr feats:
# !cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
# --minboxes 36 --maxboxes 36
# !cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
# --minboxes 50 --maxboxes 50
# !cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
# --minboxes 72 --maxboxes 72

# ## VG feats:
# !cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vg \
# --minboxes 50 --maxboxes 50

# !cd ../vilio/py*/data; ls

In [ ]:
# hm_feats.sh (E)
## Attr feats:
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
--minboxes 36 --maxboxes 36
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
--minboxes 72 --maxboxes 72
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
--minboxes 10 --maxboxes 100

## VG feats:
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vg \
--minboxes 50 --maxboxes 50
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vg \
--minboxes 10 --maxboxes 100

!cd ../vilio/py*/data; ls

Config '/vilio/py-bottom-up-attention/configs/VG-Detection/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Modifications for VG in ResNet Backbone (modeling/backbone/resnet.py):
	Using pad 0 in stem max_pool instead of pad 1.

Modifications for VG in RPN (modeling/proposal_generator/rpn.py):
	Use hidden dim 512 instead fo the same dim as Res4 (1024).

Modifications for VG in RoI heads (modeling/roi_heads/roi_heads.py):
	1. Change the stride of conv1 and shortcut in Res5.Block1 from 2 to 1.
	2. Modifying all conv2 with (padding: 1 --> 2) and (dilation: 1 --> 2).
	For more details, please check 'https://github.com/peteanderson80/bottom-up-attention/blob/master/models/vg/ResNet-101/faster_rcnn_end2end_final/test.prototxt'.

Modifications for VG in RoI heads (modeling/roi_heads/fast_rcnn.py))
	Embedding: 1601 --> 256	Linear: 2304 --> 512	Linear: 512 --> 401

faster_rcnn_from_caffe_attr.pkl: 262MB [00:11, 23.8MB/s]               
100

### copy text/image features to designated directory

In [ ]:
# # copy features to dsignated directory (O/U/V)
# !cp -r /vilio/py*/data/img/ /vilio/data/

# # delete original for memory saving
# ## delete image first
# !rm -r /vilio/py*/data/img

# ## resume
# !cp /vilio/py*/data/hm_vgattr3636.tsv /vilio/data/
# !rm /vilio/py*/data/hm_vgattr3636.tsv

# !cp /vilio/py*/data/hm_vgattr5050.tsv /vilio/data/
# !rm /vilio/py*/data/hm_vgattr5050.tsv

# !cp /vilio/py*/data/hm_vgattr7272.tsv /vilio/data/
# !rm /vilio/py*/data/hm_vgattr7272.tsv

# !cp /vilio/py*/data/hm_vg5050.tsv /vilio/data/
# !rm /vilio/py*/data/hm_vg5050.tsv

In [ ]:
# copy features to dsignated directory (E)
!cp -r /vilio/py*/data/img/ /vilio/ernie-vil/data/hm/

# delete original for memory saving
## delete image first
!rm -r /vilio/py*/data/img

## resume
!cp /vilio/py*/data/hm_vgattr3636.tsv /vilio/ernie-vil/data/hm/
!rm /vilio/py*/data/hm_vgattr3636.tsv


!cp /vilio/py*/data/hm_vgattr7272.tsv /vilio/ernie-vil/data/hm/
!rm /vilio/py*/data/hm_vgattr7272.tsv

!cp /vilio/py*/data/hm_vgattr10100.tsv /vilio/ernie-vil/data/hm/
!rm /vilio/py*/data/hm_vgattr10100.tsv

!cp /vilio/py*/data/hm_vg5050.tsv /vilio/ernie-vil/data/hm/
!rm /vilio/py*/data/hm_vg5050.tsv

!cp /vilio/py*/data/hm_vg10100.tsv /vilio/ernie-vil/data/hm/
!rm /vilio/py*/data/hm_vg10100.tsv

In [ ]:
# delete original for memory saving
## resume
!rm -r /vilio/py*/data/*

In [ ]:
# check list of data
!cd ../vilio/data; ls
!echo =========================
!cd ../vilio/ernie-vil/data/hm; ls

README.md
detectron.lmdb	  hm_vg5050.tsv       img		 train.jsonl
dev_seen.jsonl	  hm_vgattr10100.tsv  README.md
dev_unseen.jsonl  hm_vgattr3636.tsv   test_seen.jsonl
hm_vg10100.tsv	  hm_vgattr7272.tsv   test_unseen.jsonl


## additional installation

### for O/U/V

In [ ]:
# %%writefile new_req.txt
# sacremoses==0.0.43
# pandas==1.1.3
# regex==2020.4.4
# h5py==2.10.0
# filelock==3.0.10
# scipy==1.4.1
# sentencepiece~=0.1.91
# matplotlib==3.2.1
# torch==1.6.0
# tensorflow==2.3.1
# tqdm==4.45.0
# numpy==1.18.1
# six==1.14.0
# packaging==20.1
# wandb==0.10.8
# psutil==5.7.0
# requests==2.23.0
# pytorch_lightning==1.0.4
# ImageHash==4.1.0
# tokenizers~=0.9.2
# transformers==3.5.1 # Required due to some imports in the files under src/vilio/transformers
# torchvision==0.7.0
# jieba==0.42.1
# botocore==1.19.8
# spacy==2.3.2
# boto3==1.16.8
# comet_ml==3.2.5
# dataclasses==0.6
# fairseq==0.9.0
# ftfy==5.8
# fugashi==1.0.5
# ipadic==1.0.0
# lmdb==1.0.0
# Pillow==8.0.1
# py3nvml==0.2.6
# pydantic==1.7.2
# pythainlp==2.2.4
# PyYAML==5.3.1
# scikit_learn==0.23.2
# tensorboardX==2.1
# timeout_decorator==0.4.1
# torchcontrib==0.0.2

Writing new_req.txt


In [ ]:
# !pip install -r new_req.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 883 kB 7.4 MB/s 
     |████████████████████████████████| 9.5 MB 6.5 MB/s 
     |████████████████████████████████| 679 kB 67.2 MB/s 
     |████████████████████████████████| 2.9 MB 44.6 MB/s 
     |████████████████████████████████| 26.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 59.6 MB/s 
     |████████████████████████████████| 12.4 MB 30.6 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 320.4 MB 30 kB/s 
     |████████████████████████████████| 60 kB 9.9 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.7 MB 55.9 MB/s 
     |████████████████████████████████| 449 kB 71.3 MB/s 
     |████████████████████████████████| 554 kB 66.1 MB/s 
     |████████████████████████████████| 291 kB 75.1 MB/s 
     |████████████████████████████

### for E

In [35]:
%%writefile /vilio/ernie-vil/requirements.txt
# numpy set to 1.14.5 to resolve incompatibility with opencv/scipy
nltk==3.2.4
numpy==1.14.5
scipy==1.2.1
six==1.11.0
json_lines==0.5.0
lmdb==0.97
opencv-python==3.4.2.17
paddlepaddle-gpu==1.8.3.post97
pandas==1.0.5
ImageHash==4.1.0
matplotlib==3.2.1
Pillow==8.0.1
scikit_learn==0.23.2

Overwriting /vilio/ernie-vil/requirements.txt


In [36]:
# Install PaddlePaddle Requirements
!cd ../vilio/ernie-vil; pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 12.5 MB/s 
     |████████████████████████████████| 12.2 MB 12.4 MB/s 
     |████████████████████████████████| 24.8 MB 1.5 MB/s 
     |████████████████████████████████| 869 kB 49.5 MB/s 
     |████████████████████████████████| 25.0 MB 1.8 MB/s 
     |████████████████████████████████| 404.9 MB 15 kB/s 
     |████████████████████████████████| 10.1 MB 37.9 MB/s 
     |████████████████████████████████| 291 kB 51.3 MB/s 
     |████████████████████████████████| 12.4 MB 50.1 MB/s 
     |████████████████████████████████| 2.2 MB 42.7 MB/s 
     |████████████████████████████████| 6.8 MB 43.9 MB/s 
     |████████████████████████████████| 6.1 MB 46.0 MB/s 
     |████████████████████████████████| 4.4 MB 39.1 MB/s 
  Created wheel for nltk: filename=nltk-3.2.4-py3-none-any.whl size=1367723 sha256=4a0854de97823d9aaaf6ff685be27b187777f35d30afd687c116e67c4b667

In [ ]:
import os
os.kill(os.getpid(), 9)

## hm_pipeline

### download model

In [ ]:
# download the models
## O
# !wget https://biglmdiag.blob.core.windows.net/oscar/pretrained_models/large-vg-labels.zip
## U
# !wget https://convaisharables.blob.core.windows.net/uniter/pretrained/uniter-large.pt
## V
# !wget https://dl.fbaipublicfiles.com/mmf/data/models/visual_bert/visual_bert.pretrained.coco.tar.gz
## E
### EL
# !wget https://ernie-github.cdn.bcebos.com/model-ernie-vil-large-en.1.tar.gz
# !wget https://ernie-github.cdn.bcebos.com/model-ernie-vil-large-VCR-task-pre-en.1.tar.gz
### ES
!wget https://ernie-github.cdn.bcebos.com/model-ernie-vil-base-en.1.tar.gz
!wget https://ernie-github.cdn.bcebos.com/model-ernie-vil-base-VCR-task-pre-en.1.tar.gz

--2022-09-05 13:01:33--  https://ernie-github.cdn.bcebos.com/model-ernie-vil-base-en.1.tar.gz
Resolving ernie-github.cdn.bcebos.com (ernie-github.cdn.bcebos.com)... 106.122.248.35
Connecting to ernie-github.cdn.bcebos.com (ernie-github.cdn.bcebos.com)|106.122.248.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 886162849 (845M) [application/x-gzip]
Saving to: ‘model-ernie-vil-base-en.1.tar.gz’

model-ernie-vil-bas 100%[===================>] 845.11M  16.6MB/s    in 53s     

2022-09-05 13:02:28 (16.0 MB/s) - ‘model-ernie-vil-base-en.1.tar.gz’ saved [886162849/886162849]

--2022-09-05 13:02:28--  https://ernie-github.cdn.bcebos.com/model-ernie-vil-base-VCR-task-pre-en.1.tar.gz
Resolving ernie-github.cdn.bcebos.com (ernie-github.cdn.bcebos.com)... 106.122.248.35
Connecting to ernie-github.cdn.bcebos.com (ernie-github.cdn.bcebos.com)|106.122.248.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 886087681 (845M) [application/x-gzip]
Sav

In [ ]:
# copy to specified directory
## O
# !unzip large-vg-labels.zip; cp large-vg-labels/ep_20_590000/pytorch_model.bin /vilio/data
## U
# !cp uniter-large.pt /vilio/data
## V
# !tar -zxvf visual_bert.pretrained.coco.tar.gz; cp model.pth /vilio/data
## E
### EL
# !mkdir /vilio/ernie-vil/data/ernielarge; mkdir /vilio/ernie-vil/data/ernielargevcr
# !tar -zxvf model-ernie-vil-large-en.1.tar.gz; cp -r model-ernie-vil-large-en/params /vilio/ernie-vil/data/ernielarge
# !tar -zxvf model-ernie-vil-large-VCR-task-pre-en.1.tar.gz; cp -r model-ernie-vil-large-VCR-task-pre-en /vilio/ernie-vil/data/ernielargevcr
### ES
!mkdir /vilio/ernie-vil/data/erniesmall; mkdir /vilio/ernie-vil/data/erniesmallvcr
!tar -zxvf model-ernie-vil-base-en.1.tar.gz; cp -r model-ernie-vil-base-en/params /vilio/ernie-vil/data/erniesmall
!tar -zxvf model-ernie-vil-base-VCR-task-pre-en.1.tar.gz; cp -r model-ernie-vil-base-VCR-task-pre-en /vilio/ernie-vil/data/erniesmallvcr

# delete original for memory saving
## O
# !rm large-vg-labels.zip; rm -r large-vg-labels; 
## U
# !rm uniter-large.pt
## V
# !rm visual_bert.pretrained.coco.tar.gz; rm config.yaml; rm model.pth
## E
### EL
# !rm model-ernie-vil-large-en.1.tar.gz; rm -r model-ernie-vil-large-en
# !rm model-ernie-vil-large-VCR-task-pre-en.1.tar.gz; rm -r model-ernie-vil-large-VCR-task-pre-en
### ES
!rm model-ernie-vil-base-en.1.tar.gz; rm -r model-ernie-vil-base-en
!rm model-ernie-vil-base-VCR-task-pre-en.1.tar.gz; rm -r model-ernie-vil-base-VCR-task-pre-en

mkdir: cannot create directory ‘/vilio/ernie-vil/data/erniesmall’: File exists
mkdir: cannot create directory ‘/vilio/ernie-vil/data/erniesmallvcr’: File exists
model-ernie-vil-base-en/
model-ernie-vil-base-en/vocab.txt
model-ernie-vil-base-en/ernie_vil_config.base.json
model-ernie-vil-base-en/params/
model-ernie-vil-base-en/params/encoder_vlayer_3_ffn_fc_1.b_0
model-ernie-vil-base-en/params/encoder_colayer_5_vl_multi_head_att_query_fc.b_0
model-ernie-vil-base-en/params/image_text_match_fc.b_0
model-ernie-vil-base-en/params/encoder_colayer_4_vl_multi_head_att_output_fc.b_0
model-ernie-vil-base-en/params/encoder_colayer_2_post_ffn_layer_norm_bias
model-ernie-vil-base-en/params/encoder_colayer_1_multi_head_att_value_fc.w_0
model-ernie-vil-base-en/params/encoder_layer_2_post_ffn_layer_norm_bias
model-ernie-vil-base-en/params/encoder_layer_5_multi_head_att_query_fc.w_0
model-ernie-vil-base-en/params/encoder_layer_1_multi_head_att_query_fc.w_0
model-ernie-vil-base-en/params/encoder_colayer_

### run

In [ ]:
# test runs 
## O
# !cd /vilio; chmod 755 bash/training/O/hm_O.sh; bash bash/training/O/hm_O.sh 20
## U
# !cd /vilio; chmod 755 bash/training/U/hm_U.sh; bash/training/U/hm_U.sh 20
## V
# !cd /vilio; chmod 755 bash/training/V/hm_V.sh; bash/training/V/hm_V.sh 20
## E
### EL
# !cd /vilio/ernie-vil; bash bash/training/EL/hm_EL.sh 20
### ES
!cd /vilio/ernie-vil; bash bash/training/ES/hm_ES.sh 20

ストリーミング出力は最後の 5000 行に切り捨てられました。
used_time: 0.22131800651550293
feed_queue size 30
epoch: 0, progress: 0/0, step: 2010, loss: 0.179807, acc: 0.750000
steps: 2010
save_steps: 1250
20220905 14:42:35 current learning_rate:0.00001000
used_time: 0.22146129608154297
feed_queue size 30
epoch: 0, progress: 0/0, step: 2020, loss: 0.947321, acc: 0.250000
steps: 2020
save_steps: 1250
20220905 14:42:37 current learning_rate:0.00001000
used_time: 0.21059942245483398
feed_queue size 30
epoch: 0, progress: 0/0, step: 2030, loss: 0.751362, acc: 0.625000
steps: 2030
save_steps: 1250
20220905 14:42:39 current learning_rate:0.00001000
used_time: 0.20171785354614258
feed_queue size 30
epoch: 0, progress: 0/0, step: 2040, loss: 0.586395, acc: 0.500000
steps: 2040
save_steps: 1250
20220905 14:42:41 current learning_rate:0.00001000
used_time: 0.19301700592041016
feed_queue size 30
epoch: 0, progress: 0/0, step: 2050, loss: 0.667524, acc: 0.750000
steps: 2050
save_steps: 1250
20220905 14:42:43 current learning

In [ ]:
# runs
## O
# !cd /vilio; chmod 755 bash/training/O/hm_O.sh; bash bash/training/O/hm_O.sh
## U
# !cd /vilio; chmod 755 bash/training/U/hm_U.sh; bash/training/U/hm_U.sh
## V
# !cd /vilio; chmod 755 bash/training/V/hm_V.sh; bash/training/V/hm_V.sh
## E
### EL
# !cd /vilio/ernie-vil; bash bash/training/EL/hm_EL.sh
### ES
# !cd /vilio/ernie-vil; bash bash/training/ES/hm_ES.sh

## final scoring

## inference-only

### download extracted features

In [38]:
# tsv features
!kaggle datasets download muennighoff/hmtsvfeats

100% 18.1G/18.2G [02:19<00:00, 244MB/s]
100% 18.2G/18.2G [02:19<00:00, 140MB/s]


In [39]:
# tsv features
## O/U/V
# !unzip hmtsvfeats.zip -d /vilio/data/
## E
!unzip hmtsvfeats.zip -d /vilio/ernie-vil/data/hm/
# tsv features
## delete original for memory saving
!rm hmtsvfeats.zip

Archive:  hmtsvfeats.zip
  inflating: /vilio/ernie-vil/data/hm/hm_vg10100.tsv  
  inflating: /vilio/ernie-vil/data/hm/hm_vg5050.tsv  
  inflating: /vilio/ernie-vil/data/hm/hm_vgattr10100.tsv  
  inflating: /vilio/ernie-vil/data/hm/hm_vgattr3636.tsv  
  inflating: /vilio/ernie-vil/data/hm/hm_vgattr5050.tsv  
  inflating: /vilio/ernie-vil/data/hm/hm_vgattr7272.tsv  


### download pretrained model

In [33]:
!mkdir /vilio/ckpt
# ES
!kaggle datasets download muennighoff/vilioe36
!unzip -j vilioe36.zip vilioes/step_2500train/* -d /vilio/ckpt/step_2500train
!unzip -j vilioe36.zip vilioes/step_2500traindev/* -d /vilio/ckpt/step_2500traindev
!rm vilioe36.zip

100% 18.3G/18.3G [03:54<00:00, 54.5MB/s]
100% 18.3G/18.3G [03:55<00:00, 83.7MB/s]
Archive:  vilioe36.zip
  inflating: /vilio/ckpt/step_2500train/LR_DECAY_COUNTER  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.b_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.b_0_beta1_pow_acc_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.b_0_beta2_pow_acc_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.b_0_moment1_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.b_0_moment2_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.w_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.w_0_beta1_pow_acc_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.w_0_beta2_pow_acc_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.w_0_moment1_0  
  inflating: /vilio/ckpt/step_2500train/encoder_colayer_0_ffn_fc_0.w_0_moment2_0  
  

### inference

In [ ]:
!cd /vilio/ernie-vil; bash ./bash/inference/ES/hm_ES36.sh \
                            /vilio/ckpt/step_2500train \
                            /vilio/ckpt/step_2500traindev

-----------  Configuration Arguments -----------
batch_size: 8
checkpoints: checkpoints
combine: False
decay_steps: 
do_test: True
do_train: False
do_val: False
epoch: 100
ernie_config_path: ./data/erniesmall/ernie_vil_config.base.json
exp: ES36
feature_size: 2048
fusion_method: sum
hierarchical_allreduce_inter_nranks: 8
init_checkpoint: /vilio/ckpt/step_2500train
is_distributed: False
learning_rate: 0.0001
lr_decay_dict_file: 
lr_decay_ratio: 0.1
lr_scheduler: linear_warmup_decay
max_img_len: 100
max_seq_len: 128
nccl_comm_num: 1
num_features: 50
num_train_steps: 1000000
output_file: 
result_file: ./data/log
save_steps: 100
skip_steps: 10
split: dev_seen
stop_steps: 5000
subtrain: False
task_group_json: ./conf/hm/task_hm.json
task_name: hm
test_filelist: 
test_split: val
train_filelist: 
use_cuda: True
use_fast_executor: True
use_fuse: False
use_gpu: True
use_hierarchical_allreduce: False
valid_filelist: 
validation_steps: 6000
verbose: False
vocab_path: ./data/erniesmall/vocab.txt
wa

# EOS